In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

DATA_DIR = 'Data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

cap = cv2.VideoCapture(0)

alphabets = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
alphabet_idx = 0

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while alphabet_idx < len(alphabets):
        ret, frame = cap.read()

        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.putText(frame, f'Capturing {alphabets[alphabet_idx]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Hand Gesture Capture', frame)

        key = cv2.waitKey(1)
        if key == ord('s'):
            alphabet_dir = os.path.join(DATA_DIR, alphabets[alphabet_idx])
            if not os.path.exists(alphabet_dir):
                os.makedirs(alphabet_dir)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for lm in hand_landmarks.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])

                    landmarks = np.array(landmarks)
                    landmark_path = os.path.join(alphabet_dir, f'{len(os.listdir(alphabet_dir))}.npy')
                    np.save(landmark_path, landmarks)
                    print(f'Saved {landmark_path}')

        elif key == ord('n'):
            alphabet_idx += 1

        elif key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
